# Multi-Layer Perceptron (MLP)
This notebook builts an MLP for classification, same way as described in [Cepeda Humerez et al. (2019)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007290)

Hyperparameters to use:

````python
input_size = 200  # Adjust based on dataset
hidden_size = [300, 200] # 300 and 200 LTUs
output_size = 2  # Number of classes
dropout_rate = 0.5 # This wasn't specified in the paper, but choose any
learning_rate = 0.001 # Not specified in the paper
epochs = 100 # Not specified in the paper
batch_size = 16 # Not specified in the paper
````

The model architecture is in ``MLP.py``

Load the MLP model codes from ``src``

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import tqdm
from sympy import sqrt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
# Import all the functions from the 'src' directory, we import all the functions from each module so we can use them straight away
from ssa_simulation import *
from ssa_analysis import *
from ssa_classification import *
from models.MLP import MLP 
%load_ext autoreload
%autoreload 2

Example usage to train an MLP

In [34]:
# Example usage
input_size = 200  # Adjust based on dataset
hidden_size = [300, 200]
output_size = 2  # Number of classes
dropout_rate = 0.5
learning_rate = 0.001
epochs = 100
batch_size = 16

# Generate synthetic data
X_train = torch.randn(1000, input_size)
y_train = torch.randint(0, output_size, (1000,))
X_val = torch.randn(200, input_size)
y_val = torch.randint(0, output_size, (200,))

# Convert to DataLoader
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Initialize and train model
file_path = "mlp_model.pth"
model = MLP(input_size, hidden_size, output_size, dropout_rate, learning_rate)
model.train_model(train_loader, val_loader, epochs, save_path=file_path)

# Load best model and evaluate
model.load_model(file_path)
test_acc = model.evaluate(val_loader)
print(f"Final Test Accuracy: {test_acc:.4f}")

# Make predictions
X_test = torch.randn(5, input_size)
predictions = model.predict(X_test)
print("Predicted classes:", predictions)


🔄 Using device: cuda (1 GPUs available)
✅ Running on CUDA!
Epoch [1/100], Loss: 107.0392, Train Acc: 0.5150
Validation Acc: 0.4900
✅ Model saved at mlp_model.pth (Best Validation Acc: 0.4900)
Epoch [2/100], Loss: 73.6615, Train Acc: 0.5730
Validation Acc: 0.4450
Epoch [3/100], Loss: 64.6390, Train Acc: 0.5790
Validation Acc: 0.4850
Epoch [4/100], Loss: 55.5709, Train Acc: 0.6180
Validation Acc: 0.5000
✅ Model saved at mlp_model.pth (Best Validation Acc: 0.5000)
Epoch [5/100], Loss: 51.5685, Train Acc: 0.6100
Validation Acc: 0.5050
✅ Model saved at mlp_model.pth (Best Validation Acc: 0.5050)
Epoch [6/100], Loss: 46.3775, Train Acc: 0.6460
Validation Acc: 0.5050
Epoch [7/100], Loss: 42.6653, Train Acc: 0.6640
Validation Acc: 0.5200
✅ Model saved at mlp_model.pth (Best Validation Acc: 0.5200)
Epoch [8/100], Loss: 39.5790, Train Acc: 0.6890
Validation Acc: 0.5150
Epoch [9/100], Loss: 37.8717, Train Acc: 0.7020
Validation Acc: 0.5250
✅ Model saved at mlp_model.pth (Best Validation Acc: 0.52

Train the MLP using SSA data, we need to first standardise the data. If we don't, the loss will be showing as nan, which is incorrect. 

In [2]:
# Train MLP model using SSA data
output_file = 'data/mRNA_trajectories_example.csv'
X_train, X_test, y_train, y_test = load_and_split_data(output_file)
print(X_train)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 2 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [3]:
# Standardize the data 
# If your input features are too large (e.g., >1000) or too small (<0.0001), it can cause unstable training, so it's better to standardize the data.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)

[[ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 ...
 [ 0.          1.00514142 -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.          2.44105774 -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]
 [ 0.         -0.4307749  -0.30161953 ... -0.05598925 -0.09728167
  -0.07930516]]


In [5]:
# Define model parameters
input_size = X_train.shape[1]
output_size = len(set(y_train))  # Number of classes
hidden_size = [300, 200]
dropout_rate = 0.5
learning_rate = 0.001
epochs = 1000
batch_size = 32

train_loader = DataLoader(TensorDataset(
    torch.tensor(X_train, dtype=torch.float32),
    torch.tensor(y_train, dtype=torch.long)),
    batch_size=batch_size, shuffle=True
)

test_loader = DataLoader(TensorDataset(
    torch.tensor(X_test, dtype=torch.float32),
    torch.tensor(y_test, dtype=torch.long)),
    batch_size=batch_size, shuffle=False
)

model = MLP(input_size, hidden_size, output_size, dropout_rate, learning_rate)
model.train_model(train_loader, epochs=epochs)

# Evaluate MLP model
mlp_accuracy = model.evaluate(test_loader)
print(f"MLP Test Accuracy: {mlp_accuracy:.4f}")

🔄 Using device: cuda (1 GPUs available)
DEBUG: Optimizer initialized? True
✅ Running on CUDA!
Epoch [1/1000], Loss: 16.7488, Train Acc: 0.6031
Epoch [2/1000], Loss: 8.7939, Train Acc: 0.7125
Epoch [3/1000], Loss: 7.7633, Train Acc: 0.7188
Epoch [4/1000], Loss: 7.1606, Train Acc: 0.7469
Epoch [5/1000], Loss: 6.3700, Train Acc: 0.7469
Epoch [6/1000], Loss: 6.0609, Train Acc: 0.7344
Epoch [7/1000], Loss: 6.4363, Train Acc: 0.7469
Epoch [8/1000], Loss: 5.0182, Train Acc: 0.7844
Epoch [9/1000], Loss: 5.2613, Train Acc: 0.7562
Epoch [10/1000], Loss: 5.6556, Train Acc: 0.7781
Epoch [11/1000], Loss: 5.8697, Train Acc: 0.7500
Epoch [12/1000], Loss: 5.4959, Train Acc: 0.7750
Epoch [13/1000], Loss: 5.4651, Train Acc: 0.7562
Epoch [14/1000], Loss: 5.8432, Train Acc: 0.7750
Epoch [15/1000], Loss: 4.6044, Train Acc: 0.7906
Epoch [16/1000], Loss: 5.4678, Train Acc: 0.7656
Epoch [17/1000], Loss: 4.7093, Train Acc: 0.8031
Epoch [18/1000], Loss: 5.1844, Train Acc: 0.7875
Epoch [19/1000], Loss: 4.9215, T

Same as above, but getting the accuracy in a one-liner

In [6]:
# Train SVM model using SSA data
output_file = 'data/mRNA_trajectories_example.csv'
X_train, X_test, y_train, y_test = load_and_split_data(output_file)
mlp_accuracy = mlp_classifier(X_train, X_test, y_train, y_test, epochs=100)

🔄 Using device: cuda (1 GPUs available)
DEBUG: Optimizer initialized? True
✅ Running on CUDA!
Epoch [1/100], Loss: 9.7973, Train Acc: 0.6531
Epoch [2/100], Loss: 6.6727, Train Acc: 0.7406
Epoch [3/100], Loss: 5.7438, Train Acc: 0.7688
Epoch [4/100], Loss: 4.8472, Train Acc: 0.7937
Epoch [5/100], Loss: 4.9642, Train Acc: 0.7656
Epoch [6/100], Loss: 4.6204, Train Acc: 0.7906
Epoch [7/100], Loss: 4.7605, Train Acc: 0.7812
Epoch [8/100], Loss: 4.7469, Train Acc: 0.7844
Epoch [9/100], Loss: 4.5166, Train Acc: 0.8125
Epoch [10/100], Loss: 4.3355, Train Acc: 0.7937
Epoch [11/100], Loss: 4.0849, Train Acc: 0.8219
Epoch [12/100], Loss: 4.1167, Train Acc: 0.8031
Epoch [13/100], Loss: 4.2674, Train Acc: 0.8063
Epoch [14/100], Loss: 4.0178, Train Acc: 0.8187
Epoch [15/100], Loss: 4.2022, Train Acc: 0.7906
Epoch [16/100], Loss: 3.9901, Train Acc: 0.8156
Epoch [17/100], Loss: 4.2762, Train Acc: 0.8031
Epoch [18/100], Loss: 4.0971, Train Acc: 0.7875
Epoch [19/100], Loss: 4.1427, Train Acc: 0.8156
Epo